In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data=pd.read_csv('train_set.csv')
data.head()

,age,relaxation,Gtp,triglyceride,eyesight(left),ALT,height(cm),weight(kg),waist(cm),hearing(left),Urine protein,systolic,HDL,hearing(right),hemoglobin,smoking
0,40,64,10,40,1.2,10,160,50,68.0,1,1,105,70,1,11.7,0
1,60,86,111,160,0.6,55,160,65,88.0,1,3,135,55,1,14.8,0
2,40,84,28,163,1.0,16,175,75,88.0,1,1,128,48,1,15.2,1
3,40,89,12,53,1.2,17,175,60,73.0,1,1,127,64,1,15.3,1
4,55,80,34,53,0.6,24,170,70,87.1,1,1,130,85,1,14.6,1


In [5]:
X=data.drop(['smoking'],axis=1)
y=data['smoking']


In [6]:
X.head()


,age,relaxation,Gtp,triglyceride,eyesight(left),ALT,height(cm),weight(kg),waist(cm),hearing(left),Urine protein,systolic,HDL,hearing(right),hemoglobin
0,40,64,10,40,1.2,10,160,50,68.0,1,1,105,70,1,11.7
1,60,86,111,160,0.6,55,160,65,88.0,1,3,135,55,1,14.8
2,40,84,28,163,1.0,16,175,75,88.0,1,1,128,48,1,15.2
3,40,89,12,53,1.2,17,175,60,73.0,1,1,127,64,1,15.3
4,55,80,34,53,0.6,24,170,70,87.1,1,1,130,85,1,14.6


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and fit each model
bagging_model = BaggingClassifier(n_estimators=1000)
bagging_model.fit(X_train, y_train)

boosting_model = AdaBoostClassifier(n_estimators=1000)
boosting_model.fit(X_train, y_train)

random_forest_model = RandomForestClassifier(n_estimators=1000)
random_forest_model.fit(X_train, y_train)

# Make predictions
bagging_predictions = bagging_model.predict(X_test)
boosting_predictions = boosting_model.predict(X_test)
random_forest_predictions = random_forest_model.predict(X_test)

# Evaluate the models
bagging_accuracy = accuracy_score(y_test, bagging_predictions)
boosting_accuracy = accuracy_score(y_test, boosting_predictions)
random_forest_accuracy = accuracy_score(y_test, random_forest_predictions)

print(f"Bagging Accuracy: {bagging_accuracy}")
print(f"Boosting Accuracy: {boosting_accuracy}")
print(f"Random Forest Accuracy: {random_forest_accuracy}")

Bagging Accuracy: 0.7633063527103262
Boosting Accuracy: 0.7685553102771646
Random Forest Accuracy: 0.7668383615403483
